<a href="https://colab.research.google.com/github/mauricioyc/ngcf_pytorch_g61/blob/master/Neural_Graph_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from time import time
from datetime import datetime
import torch
import os
import sys

# drive path to the project
source_path = "drive/MyDrive/Machine Learning/git/ngcf_pytorch_g61"
sys.path.insert(0,source_path)

# torch cuda initialization
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0)

# Introduction

Neural Graph Collaborative Filtering (NGCF), created by [Wang et al.(2019)] (https://arxiv.org/abs/1905.08108), is a Deep Learning Recommendation algorithm with graph topology that creates user-item embeddings representation and interactions. In this article we will use [MovieLens: ML-100k dataset](https://grouplens.org/datasets/movielens/100k/) to explore [this](https://github.com/metahexane/ngcf_pytorch_g61) PyTorch implementation of the NGCF.





# Background Information

![imagem](https://drive.google.com/uc?export=view&id=1rAQFHvKvVieAI7pVmfYZPTrHuPxn16n2)

# NGCF

In a traditional collaborative filtering algorithm for ML-100k, the user-item interaction can be represented by a sparse matrix $R_{n,m}$ of liked/watched movies, where $n$ is the number of users and $m$ the movies. This matrix can be factored into two feature matrices in the form
$R_{n,m} = U_{n,f} \times I_{f,m}$, where $f$ is the feature vector size as shown in the figure 2. We can call these factors **Embeddings**.

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1uBb6pFf12tU6l29YPMBb_2YCX-T4SBbY" height="300"><figcaption>Figure 2: Matrix factorization of $R_{n,m} = U_{n,f} \times I_{f,m}$.</figcaption></center>
</br>

In the NGCF, the user-item relationship can be interpreted as a bipartite graph, shown in figure 3, in which node $u_i$ and $i_i$ is a unique user and movie embedding, respectively, and the edge represents the movies each user has seen. The graph can be created from the sparce matrix $R_{n,m}$.

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1GO5fqKaC_gRnijLjjff1nvtv4X--wMIw" ><figcaption>Figure 3: User-item bipartite graph.</figcaption></center>
</br>

In the paper, the NGCF propagates the messages from the user-item embeddings relationship over the the bipartate graph, being able to capture neighbors information depending on the connectivity order parameterized. For example, a 2nd-oder connectivity can capture the path $u1 \leftarrow i2 \leftarrow u2$ and a 3rd-order connectivity send information to $u1$ through the path $u1 \leftarrow i2 \leftarrow u2 \leftarrow i3$, being able to capture the fact that user 1 might like movie 3.

## 1: NGCF Architecture and Message Propagation

The NGCF is composed by an user and item Embeddings Layer, followed by an Embedding Propagation Layer, that receives the embeddings and calculates an recurrent update of these embeddings, and a final Prediction Layer that calculates the final prediction for a item-user pair. The layers are explained in the following.

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1C4XBYjHCQAcNYFpyT73gvsjoWWWjabaS" height="400"><figcaption>Figure 4: NGCF architecture.</figcaption></center>
</br>

### 1.1: Embedding Layer

The initial user and item embeddings are concatenated in an embedding lookup table as shown in the equation below. This embedding table is initialized using the user and item embeddings with a xavier uniform and will be optimized in an end-to-end fashion by the network.

<br>
<center>$E = [user\_embeddings \space , \space item\_embeddings] = [e_{u_1},...,e_{u_n} \space , \space e_{i_1},...,e_{i_m}]$ </center>
</br>

```python
# initialize weights
def _init_weights(self):
    print("Initializing weights...")
    weight_dict = nn.ParameterDict()

    initializer = torch.nn.init.xavier_uniform_
    
    weight_dict['user_embedding'] = nn.Parameter(initializer(torch.empty(self.n_users, self.emb_dim).to(device)))
    weight_dict['item_embedding'] = nn.Parameter(initializer(torch.empty(self.n_items, self.emb_dim).to(device)))

# ... omitted code

# creating E
def forward(self, u, i, j):
  # ... omitted code
  ego_embeddings = torch.cat([self.weight_dict['user_embedding'], self.weight_dict['item_embedding']], 0)
  # ... omitted code
```

### 1.2: Embedding Propagation Layers

The Embedding Propagation Layers receives the user-item embbedings, as well as the graph connectivety to construct the message. The message takes into consideration the current embedding state of $e_u^{0}$ and $e_i^{0}$ and updates in a recurrent form the respective embedding. In each step $l$, a $e_u^{l}$ and $e_i^{l}$ is created and concatenated to form the final features to the Predict Layer.

#### 1.2.2: Message Definition:

<center>
$\begin{equation}
  \begin{cases}
      m^{(l)}_{u \space \leftarrow \space i}  = \frac{1}{\sqrt{|\mathcal{N}_u||\mathcal{N}_i|}}\left(W^{(l)}_1e^{(l-1)}_i + W^{(l)}_2(e^{(l-1)}_i \space \odot \space e^{(l-1)}_u)\right)  \\
      m^{(l)}_{u \space \leftarrow \space u}  = W^{(l)}_1e^{(l-1)}_u \\
    \end{cases}       
\end{equation}$
<br></br>
</center>



#### 1.2.1: Message Aggregation

<center>
$e^{(l)}_{u} = \text{LeakyReLU}(m^{(l)}_{u \space \leftarrow \space u} + \sum_{i \space \in \space \mathcal{N}_u} m^{(l)}_{u \space \leftarrow \space i})$
</center>
<br></br>

#### 1.2.3: Matrix Form:

<center>
$E^{(l)} = \text{LeakyReLU}\left((\mathcal{L} + I)E^{(l-1)}W^{(l)}_1 + \mathcal{L}E^{(l-1)} \space \odot \space E^{(l-1)}W^{(l)}_2\right)$
</center>

where $\mathcal{L}$ represents the Laplacian for the user-item graph, which is formulated as:
 
<center>
$\mathcal{L} = D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$ and 
$A = \begin{equation}
  \begin{bmatrix}
      0&R \\
      R^T&0 
    \end{bmatrix}       
\end{equation}$
</center>

In the code, first we inicialize the $A$ and $\mathcal{L}$ from the train set. 

```python
# Creating A = (L + I), where R = Train_Dok_matrix
adj_mat[:self.n_users, self.n_users:] = R
adj_mat[self.n_users:, :self.n_users] = R.T

# normalize adjacency matrix
def normalized_adj_single(adj):
    rowsum = np.array(adj.sum(1))

    d_inv = np.power(rowsum, -.5).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat_inv = sp.diags(d_inv)

    norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv)
    return norm_adj.tocoo()
adj_mtx = normalized_adj_single(adj_mat) + sp.eye(adj_mat.shape[0])

# from A, create L = (A - I)
self.adj_mtx = adj_mtx
self.laplacian = adj_mtx - sp.eye(adj_mtx.shape[0])

# Create Matrix 'A', PyTorch sparse tensor of SP adjacency_mtx
self.A = self._convert_sp_mat_to_sp_tensor(self.adj_mtx)
self.L = self._convert_sp_mat_to_sp_tensor(self.laplacian)

# apply drop-out mask
A_hat = self._droupout_sparse(self.A) if self.node_dropout > 0 else self.A
L_hat = self._droupout_sparse(self.L) if self.node_dropout > 0 else self.L
```

Then, using the first embedding layer created $e^0$, a recurrent propagation calculates the following embeddings with message information $e^l_u$ and $e^l_i$, where:

<center>
$\text{side_embeddings} = (\mathcal{L} + I)E^{(l-1)}W^{(l)}_1$

$\text{side_L_embeddings} = \mathcal{L}E^{(l-1)} \space \odot \space E^{(l-1)}W^{(l)}_2$
</center>

```python
# forward pass for 'n' propagation layers
for k in range(self.n_layers):
    # weighted sum messages of neighbours
    side_embeddings = torch.sparse.mm(A_hat, ego_embeddings)
    side_L_embeddings = torch.sparse.mm(L_hat, ego_embeddings)

    # transformed sum weighted sum messages of neighbours
    sum_embeddings = torch.matmul(side_embeddings, self.weight_dict['W_gc_%d' % k]) + self.weight_dict['b_gc_%d' % k]

    # bi messages of neighbours
    bi_embeddings = torch.mul(ego_embeddings, side_L_embeddings)
    # transformed bi messages of neighbours
    bi_embeddings = torch.matmul(bi_embeddings, self.weight_dict['W_bi_%d' % k]) + self.weight_dict['b_bi_%d' % k]

    # non-linear activation 
    ego_embeddings = F.leaky_relu(sum_embeddings + bi_embeddings)
    # + message dropout
    mess_dropout_mask = nn.Dropout(self.mess_dropout)
    ego_embeddings = mess_dropout_mask(ego_embeddings)

    # normalize activation
    norm_embeddings = F.normalize(ego_embeddings, p=2, dim=1)

    all_embeddings.append(norm_embeddings)
```

### 1.3: Prediction Layer

The model prediction calculates the multiplication of $e^*_u$ by $e^*_i$, which is the logits of the user $u$ to see the movie $i$. 

<center>
$\hat{y}_NGCF = e^{* \top}_u e^*_i, \space$ where $\space e^*_u = e^{(0)}_u || \space .... \space || e^{(L)}_u \space \text{and} \space \space e^*_i = e^{(0)}_i || \space .... \space || e^{(L)}_i$
</center>

```python
all_embeddings = torch.cat(all_embeddings, 1)

# back to user/item dimension
u_g_embeddings, i_g_embeddings = all_embeddings.split([self.n_users, self.n_items], 0)

# back to user/item dimension
u_g_embeddings, i_g_embeddings = all_embeddings.split([self.n_users, self.n_items], 0)

self.u_g_embeddings = nn.Parameter(u_g_embeddings)
self.i_g_embeddings = nn.Parameter(i_g_embeddings)

u_emb = u_g_embeddings[u] # user embeddings
p_emb = i_g_embeddings[i] # positive item embeddings
n_emb = i_g_embeddings[j] # negative item embeddings

y_ui = torch.mul(u_emb, p_emb).sum(dim=1)
y_uj = torch.mul(u_emb, n_emb).sum(dim=1)
```

### 1.4: BRP Loss

```python
log_prob = (torch.log(torch.sigmoid(y_ui-y_uj))).mean()

# compute bpr-loss
bpr_loss = -log_prob
if self.reg > 0.:
    l2norm = (torch.sum(u_emb**2)/2. + torch.sum(p_emb**2)/2. + torch.sum(n_emb**2)/2.) / u_emb.shape[0]
    l2reg  = self.reg*l2norm
    bpr_loss =  -log_prob + l2reg

return bpr_loss
```

## 2: Coding


### 2.1: Data Generator

In [3]:

import random as rd
import scipy.sparse as sp
import numpy as np
[]
class Data(object):
    def __init__(self, path, batch_size):
        self.path = path
        self.batch_size = batch_size

        train_file = path + '/train.txt'
        test_file = path + '/test.txt'

        #get number of users and items
        self.n_users, self.n_items = 0, 0
        self.n_train, self.n_test = 0, 0
        self.neg_pools = {}

        self.exist_users = []

        # search train_file for max user_id/item_id
        with open(train_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    # first element is the user_id, rest are items
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    # item/user with highest number is number of items/users
                    self.n_items = max(self.n_items, max(items))
                    self.n_users = max(self.n_users, uid)
                    # number of interactions
                    self.n_train += len(items)

        # search test_file for max item_id
        with open(test_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n')
                    try:
                        items = [int(i) for i in l.split(' ')[1:]]
                    except Exception:
                        continue
                    if not items:
                        print("empyt test exists")
                        pass
                    else:
                        self.n_items = max(self.n_items, max(items))
                        self.n_test += len(items)
        # adjust counters: user_id/item_id starts at 0
        self.n_items += 1
        self.n_users += 1

        self.print_statistics()

        # create interactions/ratings matrix 'R' # dok = dictionary of keys
        print('Creating interaction matrices R_train and R_test...')
        t1 = time()
        self.R_train = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32) 
        self.R_test = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)

        self.train_items, self.test_set = {}, {}
        with open(train_file) as f_train:
            with open(test_file) as f_test:
                for l in f_train.readlines():
                    if len(l) == 0: break
                    l = l.strip('\n')
                    items = [int(i) for i in l.split(' ')]
                    uid, train_items = items[0], items[1:]
                    # enter 1 if user interacted with item
                    for i in train_items:
                        self.R_train[uid, i] = 1.
                    self.train_items[uid] = train_items

                for l in f_test.readlines():
                    if len(l) == 0: break
                    l = l.strip('\n')
                    try:
                        items = [int(i) for i in l.split(' ')]
                    except Exception:
                        continue
                    uid, test_items = items[0], items[1:]
                    for i in test_items:
                        self.R_test[uid, i] = 1.0
                    self.test_set[uid] = test_items
        print('Complete. Interaction matrices R_train and R_test created in', time() - t1, 'sec')

    # if exist, get adjacency matrix
    def get_adj_mat(self):
        try:
            t1 = time()
            adj_mat = sp.load_npz(self.path + '/s_adj_mat.npz')
            print('Loaded adjacency-matrix (shape:', adj_mat.shape,') in', time() - t1, 'sec.')

        except Exception:
            print('Creating adjacency-matrix...')
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat.npz', adj_mat)
        return adj_mat
    
    # create adjancency matrix
    def create_adj_mat(self):
        t1 = time()
        
        adj_mat = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
        adj_mat = adj_mat.tolil()
        R = self.R_train.tolil() # to list of lists

        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T
        adj_mat = adj_mat.todok()
        print('Complete. Adjacency-matrix created in', adj_mat.shape, time() - t1, 'sec.')

        t2 = time()

        # normalize adjacency matrix
        def normalized_adj_single(adj):
            rowsum = np.array(adj.sum(1))

            d_inv = np.power(rowsum, -.5).flatten()
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)

            norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv)
            return norm_adj.tocoo()

        print('Transforming adjacency-matrix to NGCF-adjacency matrix...')
        ngcf_adj_mat = normalized_adj_single(adj_mat) + sp.eye(adj_mat.shape[0])

        print('Complete. Transformed adjacency-matrix to NGCF-adjacency matrix in', time() - t2, 'sec.')
        return ngcf_adj_mat.tocsr()

    # create collections of N items that users never interacted with
    def negative_pool(self):
        t1 = time()
        for u in self.train_items.keys():
            neg_items = list(set(range(self.n_items)) - set(self.train_items[u]))
            pools = [rd.choice(neg_items) for _ in range(100)]
            self.neg_pools[u] = pools
        print('refresh negative pools', time() - t1)

    # sample data for mini-batches
    def sample(self):
        if self.batch_size <= self.n_users:
            users = rd.sample(self.exist_users, self.batch_size)
        else:
            users = [rd.choice(self.exist_users) for _ in range(self.batch_size)]

        def sample_pos_items_for_u(u, num):
            pos_items = self.train_items[u]
            n_pos_items = len(pos_items)
            pos_batch = []
            while True:
                if len(pos_batch) == num: break
                pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
                pos_i_id = pos_items[pos_id]

                if pos_i_id not in pos_batch:
                    pos_batch.append(pos_i_id)
            return pos_batch

        def sample_neg_items_for_u(u, num):
            neg_items = []
            while True:
                if len(neg_items) == num: break
                neg_id = np.random.randint(low=0, high=self.n_items,size=1)[0]
                if neg_id not in self.train_items[u] and neg_id not in neg_items:
                    neg_items.append(neg_id)
            return neg_items

        def sample_neg_items_for_u_from_pools(u, num):
            neg_items = list(set(self.neg_pools[u]) - set(self.train_items[u]))
            return rd.sample(neg_items, num)

        pos_items, neg_items = [], []
        for u in users:
            pos_items += sample_pos_items_for_u(u, 1)
            neg_items += sample_neg_items_for_u(u, 1)

        return users, pos_items, neg_items

    def get_num_users_items(self):
        return self.n_users, self.n_items

    def print_statistics(self):
        print('n_users=%d, n_items=%d' % (self.n_users, self.n_items))
        print('n_interactions=%d' % (self.n_train + self.n_test))
        print('n_train=%d, n_test=%d, sparsity=%.5f' % (self.n_train, self.n_test, (self.n_train + self.n_test)/(self.n_users * self.n_items)))


### 2.2: Model

In [4]:
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn.functional as F
import scipy.sparse as sp

from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class NGCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim, layers, reg, node_dropout, mess_dropout,
        adj_mtx):
        super().__init__()

        # initialize Class attributes
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.adj_mtx = adj_mtx
        self.laplacian = adj_mtx - sp.eye(adj_mtx.shape[0])
        self.reg = reg
        self.layers = layers
        self.n_layers = len(self.layers)
        self.node_dropout = node_dropout
        self.mess_dropout = mess_dropout

        #self.u_g_embeddings = nn.Parameter(torch.empty(n_users, emb_dim+np.sum(self.layers)))
        #self.i_g_embeddings = nn.Parameter(torch.empty(n_items, emb_dim+np.sum(self.layers)))

        # Initialize weights
        self.weight_dict = self._init_weights()
        print("Weights initialized.")

        # Create Matrix 'A', PyTorch sparse tensor of SP adjacency_mtx
        self.A = self._convert_sp_mat_to_sp_tensor(self.adj_mtx)
        self.L = self._convert_sp_mat_to_sp_tensor(self.laplacian)

    # initialize weights
    def _init_weights(self):
        print("Initializing weights...")
        weight_dict = nn.ParameterDict()

        initializer = torch.nn.init.xavier_uniform_
        
        weight_dict['user_embedding'] = nn.Parameter(initializer(torch.empty(self.n_users, self.emb_dim).to(device)))
        weight_dict['item_embedding'] = nn.Parameter(initializer(torch.empty(self.n_items, self.emb_dim).to(device)))

        weight_size_list = [self.emb_dim] + self.layers

        for k in range(self.n_layers):
            weight_dict['W_gc_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_gc_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
            
            weight_dict['W_bi_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_bi_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
           
        return weight_dict

    # convert sparse matrix into sparse PyTorch tensor
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res

    # apply node_dropout
    def _droupout_sparse(self, X):
        """
        Drop individual locations in X
        
        Arguments:
        ---------
        X = adjacency matrix (PyTorch sparse tensor)
        dropout = fraction of nodes to drop
        noise_shape = number of non non-zero entries of X
        """
        
        node_dropout_mask = ((self.node_dropout) + torch.rand(X._nnz())).floor().bool().to(device)
        i = X.coalesce().indices()
        v = X.coalesce()._values()
        i[:,node_dropout_mask] = 0
        v[node_dropout_mask] = 0
        X_dropout = torch.sparse.FloatTensor(i, v, X.shape).to(X.device)

        return  X_dropout.mul(1/(1-self.node_dropout))

    def forward(self, u, i, j):
        """
        Computes the forward pass
        
        Arguments:
        ---------
        u = user
        i = positive item (user interacted with item)
        j = negative item (user did not interact with item)
        """
        # apply drop-out mask
        A_hat = self._droupout_sparse(self.A) if self.node_dropout > 0 else self.A
        L_hat = self._droupout_sparse(self.L) if self.node_dropout > 0 else self.L

        ego_embeddings = torch.cat([self.weight_dict['user_embedding'], self.weight_dict['item_embedding']], 0)

        all_embeddings = [ego_embeddings]

        # forward pass for 'n' propagation layers
        for k in range(self.n_layers):

            # weighted sum messages of neighbours
            side_embeddings = torch.sparse.mm(A_hat, ego_embeddings)
            side_L_embeddings = torch.sparse.mm(L_hat, ego_embeddings)

            # transformed sum weighted sum messages of neighbours
            sum_embeddings = torch.matmul(side_embeddings, self.weight_dict['W_gc_%d' % k]) + self.weight_dict['b_gc_%d' % k]

            # bi messages of neighbours
            bi_embeddings = torch.mul(ego_embeddings, side_L_embeddings)
            # transformed bi messages of neighbours
            bi_embeddings = torch.matmul(bi_embeddings, self.weight_dict['W_bi_%d' % k]) + self.weight_dict['b_bi_%d' % k]

            # non-linear activation 
            ego_embeddings = F.leaky_relu(sum_embeddings + bi_embeddings)
            # + message dropout
            mess_dropout_mask = nn.Dropout(self.mess_dropout)
            ego_embeddings = mess_dropout_mask(ego_embeddings)

            # normalize activation
            norm_embeddings = F.normalize(ego_embeddings, p=2, dim=1)

            all_embeddings.append(norm_embeddings)

        all_embeddings = torch.cat(all_embeddings, 1)
        
        # back to user/item dimension
        u_g_embeddings, i_g_embeddings = all_embeddings.split([self.n_users, self.n_items], 0)

        self.u_g_embeddings = nn.Parameter(u_g_embeddings)
        self.i_g_embeddings = nn.Parameter(i_g_embeddings)
        
        u_emb = u_g_embeddings[u] # user embeddings
        p_emb = i_g_embeddings[i] # positive item embeddings
        n_emb = i_g_embeddings[j] # negative item embeddings

        y_ui = torch.mul(u_emb, p_emb).sum(dim=1)
        y_uj = torch.mul(u_emb, n_emb).sum(dim=1)
        log_prob = (torch.log(torch.sigmoid(y_ui-y_uj))).mean()

        # compute bpr-loss
        bpr_loss = -log_prob
        if self.reg > 0.:
            l2norm = (torch.sum(u_emb**2)/2. + torch.sum(p_emb**2)/2. + torch.sum(n_emb**2)/2.) / u_emb.shape[0]
            l2reg  = self.reg*l2norm
            bpr_loss =  -log_prob + l2reg

        return bpr_loss

### 2.3: Utils

In [5]:
import numpy as np
import torch

def early_stopping(log_value, best_value, stopping_step, flag_step, expected_order='asc'):
    """
    Check if early_stopping is needed
    Function copied from original code
    """
    assert expected_order in ['asc', 'des']
    if (expected_order == 'asc' and log_value >= best_value) or (expected_order == 'des' and log_value <= best_value):
        stopping_step = 0
        best_value = log_value
    else:
        stopping_step += 1

    if stopping_step >= flag_step:
        print("Early stopping at step: {} log:{}".format(flag_step, log_value))
        should_stop = True
    else:
        should_stop = False

    return best_value, stopping_step, should_stop

def train(model, data_generator, optimizer):
    """
    Train the model PyTorch style

    Arguments:
    ---------
    model: PyTorch model
    data_generator: Data object
    optimizer: PyTorch optimizer
    """
    model.train()
    n_batch = data_generator.n_train // data_generator.batch_size + 1
    running_loss=0
    for _ in range(n_batch):
        u, i, j = data_generator.sample()
        optimizer.zero_grad()
        loss = model(u,i,j)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss

def split_matrix(X, n_splits=100):
    """
    Split a matrix/Tensor into n_folds (for the user embeddings and the R matrices)

    Arguments:
    ---------
    X: matrix to be split
    n_folds: number of folds

    Returns:
    -------
    splits: split matrices
    """
    splits = []
    chunk_size = X.shape[0] // n_splits
    for i in range(n_splits):
        start = i * chunk_size
        end = X.shape[0] if i == n_splits - 1 else (i + 1) * chunk_size
        splits.append(X[start:end])
    return splits

def compute_ndcg_k(pred_items, test_items, test_indices, k):
    """
    Compute NDCG@k
    
    Arguments:
    ---------
    pred_items: binary tensor with 1s in those locations corresponding to the predicted item interactions
    test_items: binary tensor with 1s in locations corresponding to the real test interactions
    test_indices: tensor with the location of the top-k predicted items
    k: k'th-order 

    Returns:
    -------
    NDCG@k
    """
    r = (test_items * pred_items).gather(1, test_indices)
    f = torch.from_numpy(np.log2(np.arange(2, k+2))).float().cuda()
    dcg = (r[:, :k]/f).sum(1)
    dcg_max = (torch.sort(r, dim=1, descending=True)[0][:, :k]/f).sum(1)
    ndcg = dcg/dcg_max
    ndcg[torch.isnan(ndcg)] = 0
    return ndcg


def eval_model(u_emb, i_emb, Rtr, Rte, k):
    """
    Evaluate the model
    
    Arguments:
    ---------
    u_emb: User embeddings
    i_emb: Item embeddings
    Rtr: Sparse matrix with the training interactions
    Rte: Sparse matrix with the testing interactions
    k : kth-order for metrics
    
    Returns:
    --------
    result: Dictionary with lists correponding to the metrics at order k for k in Ks
    """
    # split matrices
    ue_splits = split_matrix(u_emb)
    tr_splits = split_matrix(Rtr)
    te_splits = split_matrix(Rte)

    recall_k, ndcg_k= [], []
    # compute results for split matrices
    for ue_f, tr_f, te_f in zip(ue_splits, tr_splits, te_splits):

        scores = torch.mm(ue_f, i_emb.t())

        test_items = torch.from_numpy(te_f.todense()).float().cuda()
        non_train_items = torch.from_numpy(1-(tr_f.todense())).float().cuda()
        scores = scores * non_train_items

        _, test_indices = torch.topk(scores, dim=1, k=k)
        pred_items = torch.zeros_like(scores).float()
        pred_items.scatter_(dim=1,index=test_indices,value=torch.tensor(1.0).cuda())

        topk_preds = torch.zeros_like(scores).float()
        topk_preds.scatter_(dim=1,index=test_indices[:, :k],value=torch.tensor(1.0))

        TP = (test_items * topk_preds).sum(1)
        rec = TP/test_items.sum(1)
        ndcg = compute_ndcg_k(pred_items, test_items, test_indices, k)

        recall_k.append(rec)
        ndcg_k.append(ndcg)

    return torch.cat(recall_k).mean(), torch.cat(ndcg_k).mean()

### 2.4: Parameter and Base Initialization

In [6]:
# Parameters Inicialization

data_dir = source_path+'/data/'
dataset = 'ml-100k'
batch_size = 1024
layers = eval('[64,64]')
emb_dim = 64
lr = 0.0001
reg = 1e-5
mess_dropout = 0.1
node_dropout = 0.
k = 10
n_epochs = 400
eval_N = 1
save_results = 1
results_dir = 'results'

In [7]:
data_generator = Data(path=data_dir + dataset, batch_size=batch_size)
adj_mtx = data_generator.get_adj_mat()

model = NGCF(data_generator.n_users, 
              data_generator.n_items,
              emb_dim,
              layers,
              reg,
              node_dropout,
              mess_dropout,
              adj_mtx)

n_users=943, n_items=1682
n_interactions=100000
n_train=80064, n_test=19936, sparsity=0.06305
Creating interaction matrices R_train and R_test...
Complete. Interaction matrices R_train and R_test created in 1.0580151081085205 sec
Loaded adjacency-matrix (shape: (2625, 2625) ) in 0.01728987693786621 sec.
Initializing weights...


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


Weights initialized.


### 2.5: Training

In [ ]:
if use_cuda:
    model = model.cuda()

# current best metric
cur_best_metric = 0

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Set values for early stopping
cur_best_loss, stopping_step, should_stop = 1e3, 0, False
today = datetime.now()

print("Start at " + str(today))
print("Using " + str(device) + " for computations")
print("Params on CUDA: " + str(next(model.parameters()).is_cuda))

results = {"Epoch": [],
            "Loss": [],
            "Recall": [],
            "NDCG": [],
            "Training Time": []}

for epoch in range(n_epochs):

    t1 = time()
    loss = train(model, data_generator, optimizer)
    training_time = time()-t1
    print("Epoch: {}, Training time: {:.2f}s, Loss: {:.4f}".
        format(epoch, training_time, loss))

    # print test evaluation metrics every N epochs (provided by eval_N)
    if epoch % eval_N  == (eval_N - 1):
        with torch.no_grad():
            t2 = time()
            recall, ndcg = eval_model(model.u_g_embeddings.detach(),
                                      model.i_g_embeddings.detach(),
                                      data_generator.R_train,
                                      data_generator.R_test,
                                      k)
        print(
            "Evaluate current model:\n",
            "Epoch: {}, Validation time: {:.2f}s".format(epoch, time()-t2),"\n",
            "Loss: {:.4f}:".format(loss), "\n",
            "Recall@{}: {:.4f}".format(k, recall), "\n",
            "NDCG@{}: {:.4f}".format(k, ndcg)
            )

        cur_best_metric, stopping_step, should_stop = \
        early_stopping(recall, cur_best_metric, stopping_step, flag_step=5)

        # save results in dict
        results['Epoch'].append(epoch)
        results['Loss'].append(loss)
        results['Recall'].append(recall.item())
        results['NDCG'].append(ndcg.item())
        results['Training Time'].append(training_time)
    else:
        # save results in dict
        results['Epoch'].append(epoch)
        results['Loss'].append(loss)
        results['Recall'].append(None)
        results['NDCG'].append(None)
        results['Training Time'].append(training_time)

    if should_stop == True: break

Start at 2021-01-30 22:18:15.919897
Using cuda for computations
Params on CUDA: True


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


Epoch: 0, Training time: 3.97s, Loss: 54.4709
Evaluate current model:
 Epoch: 0, Validation time: 1.12s 
 Loss: 54.4709: 
 Recall@10: 0.0102 
 NDCG@10: 0.1195


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


Epoch: 1, Training time: 3.99s, Loss: 53.9444
Evaluate current model:
 Epoch: 1, Validation time: 1.13s 
 Loss: 53.9444: 
 Recall@10: 0.0114 
 NDCG@10: 0.1164


...

Epoch: 14, Training time: 4.03s, Loss: 29.8741
Evaluate current model:
 Epoch: 14, Validation time: 1.12s 
 Loss: 29.8741: 
 Recall@10: 0.0562 
 NDCG@10: 0.2798


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


Epoch: 15, Training time: 3.90s, Loss: 29.4283
Evaluate current model:
 Epoch: 15, Validation time: 1.13s 
 Loss: 29.4283: 
 Recall@10: 0.0754 
 NDCG@10: 0.3111


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


Epoch: 16, Training time: 3.92s, Loss: 29.2288


### 2.6 Results

In [ ]:
# save
if save_results:
    date = today.strftime("%d%m%Y_%H%M")

    # save model as .pt file
    if os.path.isdir("./models"):
        torch.save(model.state_dict(), "./models/" + str(date) + "_" + modelname + "_" + dataset + ".pt")
    else:
        os.mkdir("./models")
        torch.save(model.state_dict(), "./models/" + str(date) + "_" + modelname + "_" + dataset + ".pt")

    # save results as pandas dataframe
    results_df = pd.DataFrame(results)
    results_df.set_index('Epoch', inplace=True)
    if os.path.isdir("./results"):
        results_df.to_csv("./results/" + str(date) + "_" + modelname + "_" + dataset + ".csv")
    else:
        os.mkdir("./results")
        results_df.to_csv("./results/" + str(date) + "_" + modelname + "_" + dataset + ".csv")
    # plot loss
    results_df['Loss'].plot(figsize=(12,8), title='Loss')